# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
#import csv
df_path = "../output_data/cities.csv"

# Read Purchasing File and store into Pandas data frame
weather_df = pd.read_csv(df_path, index_col = 'Unnamed: 0')

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Avarua,13,CK,1587315342,83,-21.21,-159.78,77.00,4.70
1,Busselton,97,AU,1587315342,59,-33.65,115.33,55.99,14.70
2,Vardø,75,NO,1587315343,74,70.37,31.11,37.40,10.29
3,Qaanaaq,68,GL,1587315343,89,77.48,-69.36,17.53,8.03
4,Rikitea,59,PF,1587315343,70,-23.12,-134.97,76.75,7.87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#configure gmaps
gmaps.configure(api_key=g_key)

#use lat, lng, humidity
coord_ls = weather_df[['Lat','Lng']]
hum_ls = weather_df['Humidity'].astype(float)

map1 = gmaps.figure()

layer1 = gmaps.heatmap_layer(coord_ls, weights=hum_ls, dissipating=False, point_radius = 1)

map1.add_layer(layer1)

map1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#ideal conditions: domestic, not too cloudy or humid or hot or cold either
hotel_df = weather_df.loc[weather_df['Country'] == 'US']
hotel_df = hotel_df.loc[hotel_df['Cloudiness'] <= 50]
hotel_df = hotel_df.loc[hotel_df['Humidity'] <= 75]
hotel_df = hotel_df.loc[hotel_df['Max Temp'] <= 74]
hotel_df = hotel_df.loc[hotel_df['Max Temp'] >= 49]

hotel_df.count()

City          16
Cloudiness    16
Country       16
Date          16
Humidity      16
Lat           16
Lng           16
Max Temp      16
Wind Speed    16
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotel_df['Hotel Name'] = ""

hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Sterling,1,US,1587315344,38,39.01,-77.43,63.00,10.29,
1,Albany,1,US,1587315055,28,42.60,-73.97,60.01,5.01,
2,Jamestown,1,US,1587315128,50,42.10,-79.24,51.80,18.34,
3,Hilo,40,US,1587315192,68,19.73,-155.09,66.99,8.05,
4,Bethel,1,US,1587315583,43,41.37,-73.41,59.00,18.34,


In [45]:
link_root = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# location reqs for API
lat = hotel_df['Lat']
lng = hotel_df['Lng']

#reqs for API to search for hotels with 5000 meters.
rqmnts = {
    'radius': 5000,
    'rankby': 'prominence',
    'key': g_key,
    'keyword': 'hotel'
}

#Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    
    glat = row['Lat']
    glng = row['Lng']
    
    rqmnts['location'] = f'{glat},{glng}'
    
    goog_rsp = requests.get(link_root,params=rqmnts).json()
    
    gg_result = goog_rsp['results']
    
    #Store the first Hotel result into the DataFrame.
    try:
        hotel_df.loc[index, 'Hotel Name'] = gg_result[0]['name']
    
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "None Available"

hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Sterling,1,US,1587315344,38,39.01,-77.43,63.00,10.29,Best Western Dulles Airport Inn
1,Albany,1,US,1587315055,28,42.60,-73.97,60.01,5.01,None Available
2,Jamestown,1,US,1587315128,50,42.10,-79.24,51.80,18.34,Chautauqua Harbor Hotel
3,Hilo,40,US,1587315192,68,19.73,-155.09,66.99,8.05,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
4,Bethel,1,US,1587315583,43,41.37,-73.41,59.00,18.34,Best Western Danbury/Bethel


In [52]:
#removing unavailable hotels for marker placement
mark_hotels = hotel_df.loc[hotel_df['Hotel Name'] != 'None Available']

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in mark_hotels.iterrows()]
locations = mark_hotels[["Lat", "Lng"]]

# Add marker layer ontop of heat map
mark_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

map2 = map1

map2.add_layer(mark_layer)

map2

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Sterling,1,US,1587315344,38,39.01,-77.43,63.00,10.29,Best Western Dulles Airport Inn
1,Albany,1,US,1587315055,28,42.60,-73.97,60.01,5.01,None Available
2,Jamestown,1,US,1587315128,50,42.10,-79.24,51.80,18.34,Chautauqua Harbor Hotel
3,Hilo,40,US,1587315192,68,19.73,-155.09,66.99,8.05,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
4,Bethel,1,US,1587315583,43,41.37,-73.41,59.00,18.34,Courtyard by Marriott Danbury
